In [1]:
import torch
import matplotlib.pyplot as plt
from utils.defects import S_cal
import numpy as np
import pickle
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import torch
import torch.nn as nn
import tensorboard

from kinetic_solver import KineticSolver, KineticData
from stable_baselines3 import PPO
from utils.model import DownSampleConv
from utils.defects import theta_cal, S_cal
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.policies import ActorCriticCnnPolicy
from kinetic_solver import KineticSolver, KineticData
from nematic_env import ActiveNematicEnv

# 初始化物理仿真器
geo_params = {
    'N': 256,
    'Nth': 256,
    'L': 20
}
flow_params = {
    'dT': 0.3,
    'dR': 0.3,
    'alpha': -10,
    'beta': 1.0,
    'zeta': 2,
    'V0': 0.0
}
simu_params = {
    'dt': 0.0004,
    'seed': 1234,
    'inner_steps': 6,
    'outer_steps': 2
}
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

solver_paras = (geo_params, flow_params, simu_params)

In [ ]:
solver_paras = (geo_params, flow_params, simu_params)

solver = KineticSolver(*solver_paras, device=device)
data_path = '/home/hou63/pj2/Nematic_RL/datas/simulation_data_test.pkl'
# simulation_data = KineticData(*solver.initialize2_pytorch(seed=918), solver.simu_args)
simulation_data = KineticData.loader(data_path)
# simulation_data = solver.preloop_kinetic(simulation_data, num_itr=32000)
print(simulation_data)

simulation_data = simulation_data.loader(data_path)
env = ActiveNematicEnv(solver_paras, solver=solver,
                        simulation_data=simulation_data, device=device,
                        data_path=data_path)


In [ ]:
print(env._calculate_reward())

d11, d12 = simulation_data.get_D()
S = S_cal(d11.cpu().data.numpy(), d12.cpu().data.numpy())
print(S.mean())
d11 = d11.cpu().data.numpy()
d12 = d12.cpu().data.numpy()
theta = 0.5 * np.arctan2(2*d12, 2*d11-1)
# plot S

cost, sint = np.cos(theta), np.sin(theta)

fig, ax1 = plt.subplots(1,3, figsize=(15, 5))
ax1[0].streamplot(np.arange(0, theta.shape[0]), np.arange(0, theta.shape[1]),
                cost, sint, arrowsize=0.5, color='r',
                density=4, linewidth=0.5)

ax1[1].imshow(d11)
# set colorbar

ax1[2].imshow(S)
plt.show()

In [ ]:
check_env(env)

In [ ]:
model = PPO(
    ActorCriticCnnPolicy, env, verbose=1, device=device,
      tensorboard_log="/home/hou63/pj2/Nematic_RL/logs2")
model.learn(total_timesteps=2)

# 关闭环境
env.close()

In [ ]:
# save model
model.save("/home/hou63/pj2/Nematic_RL/models/ppo_nematic")
